In [6]:
from elasticsearch import Elasticsearch

In [7]:
def setting():
    settings ={
        "settings": {
            "analysis": {
                "filter": {
                    "synonyms_filter": { # 同義語フィルターの定義
                        "type": "synonym",
                        "synonyms": [ #同義語リストの定義 (今は空の状態)
                            ]
                    }
                },
                "tokenizer": {
                    "sudachi_tokenizer": {
                        "type": "sudachi_tokenizer",
                        "discard_punctuation": True,
                        "sudachi_split":"search",
                        "resources_path": "/usr/share/elasticsearch/config/sudachi",
                        "settings_path": "/usr/share/elasticsearch/config/sudachi/sudachi_fulldict.json"
                    }
                },
                "analyzer": {
                    "sudachi_analyzer": {
                        "char_filter": [
                            "icu_normalizer", # 文字単位の正規化
                            "kuromoji_iteration_mark" # 繰り返し文字の正規化
                        ],
                        "filter": [
                            "synonyms_filter", # 同義語展開
                            # "kuromoji_baseform", # 活用語の原型化
                            # "kuromoji_part_of_speech", # 不要品詞の除去
                            # "ja_stop", #不要単語の除去
                            "kuromoji_number", # 数字の正規化
                            "kuromoji_stemmer" #長音の正規化
                        ],
                        "tokenizer": "sudachi_tokenizer",
                        "type": "custom"
                    }
                }
            }
        },
      "mappings":{
            "properties": {
                "page_id": {
                    "type": "integer"
                },
                "title": {
                    "type":"keyword"
                },
                "text": {
                    "analyzer": "sudachi_analyzer",
                    "type": "text"
                },
                "sudachi" :{
                    "type": "text"
                }
                }
      }
    }
    
    return settings

In [8]:
%%time
jp_index = "test"
es = Elasticsearch("elasticsearch-sudachi")
if es.indices.exists(index = jp_index):
    print(f"{jp_index}を更新します。")
    es.indices.delete(index = jp_index)

Result = es.indices.create(index=jp_index, body=setting())
if Result["acknowledged"] == True:
    print(f'create index【{Result["index"]}】')

testを更新します。
create index【test】
CPU times: user 4.91 ms, sys: 95 µs, total: 5 ms
Wall time: 170 ms


In [9]:
%%time
import io
import json
import re
import glob

paths = glob.glob("./dataset/*/*")
for path in paths:
#     print(path)
    data = io.open(path,mode = "r").readlines()
    for line in data:
        json_data = json.loads(line)
        text = json_data["text"]
        page_id = json_data["id"]
        # タイトル抽出
        title_data = re.findall("^(.*)\n*",text)
        print(f"\r{title_data}",end="")
        # テキスト抽出
        ## テキスト中のタイトル削除
        text_data = re.sub("^.*\n*", "", text)
        ## テキスト終わりの改行削除
        text_data = re.sub("\n+$", "", text_data)
        ## テキスト中の改行を[_#_]に変換
        text_data = re.sub("\n+", "_#_", text_data)
        text_data = text_data.split('_#_')
        for num, original in enumerate(text_data, 1):
            body = {"text":original, "analyzer":"sudachi_analyzer"}
            data = es.indices.analyze(index=jp_index, body=body)["tokens"]
            sudachi_list = ""
            for i in data:
                sudachi_list += i["token"] + " "
            body = {"page_id": page_id, "title": title_data, "text": original, "sudachi":sudachi_list}
    #     body = {"title": title_data, "text": i}

            es.index(index=jp_index, id=f"{page_id}.{num}", body=body)

['アンダーリップ']CPU times: user 7h 2min 2s, sys: 36min 20s, total: 7h 38min 22s24 (Video)"]'] Shibuya C.C.Lemon Hall']tack of the Evil, Mutant, Alien, Flesh Eating, Hellbound, Zombified Living Dead Part 2: In Shocking 2-D']
Wall time: 23h 11min 17s


In [10]:
result = es.search(
            index=jp_index,
            body = {
                "query": {
                    "term": {
                        "title": "こどもの文化"
                    }
                }
            }
)

In [11]:
result

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 14.482862,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '59.1',
    '_score': 14.482862,
    '_source': {'page_id': '59',
     'title': ['こどもの文化'],
     'text': 'こどもの文化（こどものぶんか）は、子供の遊びや行事を通じて形成される子供独自の文化である。',
     'sudachi': 'こどもの文化 こども の ぶん か は 子供の遊び や 行事 を 通じ て 形成 さ れる 子供 独自 の 文化 で ある '}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '59.2',
    '_score': 14.482862,
    '_source': {'page_id': '59',
     'title': ['こどもの文化'],
     'text': '未就学あるいは義務教育段階である子供が、子供ら独自で、あるいは保護者や教師をはじめとする大人からの示唆や指導を受けながら形成し、その多くは以下に挙げるような、子供の形成過程で使われるさまざまな道具（おもちゃ）により、創造性、想像力など脳の発達と共に、文化的意味合いを持つものだと指摘できる。',
     'sudachi': '未就学 あるいは 義務教育 段階 で ある 子供 が 子供 ら 独自 で あるいは 保護者 や 教師 を はじめ と する 大人 から の 示唆 や 指導 を 受け ながら 形成 し その 多く は 以下 に 挙げる よう な 子供 の 形成 過程 で 使わ れる さまざま な 道具 おもちゃ に より 創造性 想像力 など 脳 の 発達 と 共 に 文化的 意味合い を 持つ もの だ

In [35]:
print("---------json----------")
print(json_data)
print("---------title----------")
print(title_data)
print("---------texts----------")
# print(text_data)
for num, i in enumerate(text_data, 1):
    body = {"page_id": page_id, "title": title_data, "text": i}
#     body = {"title": title_data, "text": i}

    es.index(index=jp_index, id=f"{page_id}.{num}", body=body)

---------json----------
{'id': '5', 'url': 'https://ja.wikipedia.org/wiki?curid=5', 'title': 'アンパサンド', 'text': 'アンパサンド\n\nアンパサンド (&、英語名：) とは並立助詞「…と…」を意味する記号である。ラテン語の の合字で、Trebuchet MSフォントでは、と表示され "et" の合字であることが容易にわかる。ampersa、すなわち "and per se and"、その意味は"and [the symbol which] by itself [is] and"である。\n\nその使用は1世紀に遡ることができ、5世紀中葉から現代に至るまでの変遷がわかる。\nZ に続くラテン文字アルファベットの27字目とされた時期もある。\n\nアンパサンドと同じ役割を果たす文字に「のet」と呼ばれる、数字の「7」に似た記号があった(, U+204A)。この記号は現在もゲール文字で使われている。\n\n記号名の「アンパサンド」は、ラテン語まじりの英語「& はそれ自身 "and" を表す」(& per se and) のくずれた形である。英語以外の言語での名称は多様である。\n\n日常的な手書きの場合、欧米でアンパサンドは「ε」に縦線を引く単純化されたものが使われることがある。\n\nまた同様に、「t」または「+（プラス）」に輪を重ねたような、無声歯茎側面摩擦音を示す発音記号「」のようなものが使われることもある。\n\nプログラミング言語では、C など多数の言語で AND 演算子として用いられる。以下は C の例。\nPHPでは、変数宣言記号（$）の直前に記述することで、参照渡しを行うことができる。\n\nBASIC 系列の言語では文字列の連結演算子として使用される。codice_4 は codice_5 を返す。また、主にマイクロソフト系では整数の十六進表記に codice_6 を用い、codice_7 （十進で15）のように表現する。\n\nSGML、XML、HTMLでは、アンパサンドを使ってSGML実体を参照する。\n'}
---------title----------
['アンパサンド']
---------texts----------


In [36]:
result = es.search(
            index=jp_index,
            body={"query": {"match_all": {}}}
)

In [37]:
result

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 11, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '5.1',
    '_score': 1.0,
    '_source': {'page_id': '5',
     'title': ['アンパサンド'],
     'text': 'アンパサンド (&、英語名：) とは並立助詞「…と…」を意味する記号である。ラテン語の の合字で、Trebuchet MSフォントでは、と表示され "et" の合字であることが容易にわかる。ampersa、すなわち "and per se and"、その意味は"and [the symbol which] by itself [is] and"である。'}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '5.2',
    '_score': 1.0,
    '_source': {'page_id': '5',
     'title': ['アンパサンド'],
     'text': 'その使用は1世紀に遡ることができ、5世紀中葉から現代に至るまでの変遷がわかる。'}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '5.3',
    '_score': 1.0,
    '_source': {'page_id': '5',
     'title': ['アンパサンド'],
     'text': 'Z に続くラテン文字アルファベットの27字目とされた時期もある。'}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '5.4',
    '_score': 1.0,
    '_source': {'p